Import libraries

In [1]:
%load_ext autoreload
%autoreload 2

#%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from datetime import datetime, timedelta
from ipywidgets import interact
from tqdm.notebook import tqdm
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import display
from scipy import stats
import warnings
import ipywidgets as widgets
pd.options.mode.chained_assignment = None 
from datetime import timezone
import datetime
from suntime import Sun
import pytz
from matplotlib.colors import LinearSegmentedColormap
from sklearn.datasets import make_classification
from sklearn.cluster import KMeans
import matplotlib.ticker as mticker

Read CSV

In [2]:
#Run this block for dataset D2
dataset = pd.read_csv("datasetD2.csv")

dataset['datetime'] = dataset['Date'] + " " + dataset['Time']
dataset['datetime'] = pd.to_datetime(dataset['datetime'], format='%d-%m-%Y %H:%M:%S')
dataset['tagID'] = dataset['ID']
dataset['event'] = dataset['Direction'].apply(lambda x: 'exiting' if x == "S" else 'entering')
dataset = dataset[dataset['Treatment'] == "Control"]
cols = range(7)
dataset.drop(dataset.columns[cols],axis=1,inplace=True)

lat = 18.47
lon = -66.12
tz = pytz.timezone('America/Puerto_Rico')

In [3]:
#Run this block for dataset D1
dataset = pd.read_csv("datasetD1.csv")

dataset['datetime'] = pd.to_datetime(dataset['datetime'], format='mixed')

lat = 18.47
lon = -66.12
tz = pytz.timezone('America/Puerto_Rico')

In [4]:
dataset

,tagID,datetime,event
0,1621,2017-06-21 08:00:10.850,entering
1,1621,2017-06-21 08:10:33.850,exiting
2,1621,2017-06-21 08:24:48.250,entering
3,1621,2017-06-21 08:51:23.450,entering
4,1621,2017-06-21 08:55:48.300,exiting
...,...,...,...
1148,661,2017-06-22 17:09:08.250,entering
1149,2258,2017-06-22 11:49:10.700,entering
1150,2258,2017-06-22 12:07:42.750,exiting
1151,2258,2017-06-22 16:14:06.350,entering


Data cleaning functions

Separate entry/exit events and convert into flights, takes entry/exit dataset
for a single bee and returns a table.

In [5]:
#for clean datasets
#max time btwn 2 leavings
t0 = 300
#max time btwn 2 enterings
t1 = 3600
#max time of trip
t3 = 21600


def classifyLoc(dataset):

    loc = []
    start = []
    end = []
    bee = dataset
    
    for i in range(len(bee)-1):
        cur = bee.iloc[i]
        next = bee.iloc[i+1]
        if cur['event'] == "entering" and next['event'] == "exiting":
            loc.append("Inside")
        elif cur['event'] == "exiting" and next['event'] == "entering":
            if (next['datetime'] - cur['datetime']).total_seconds() > t3:
                loc.append("Inside")
            else:
                loc.append("Outside")
        else:
            loc.append("Inside")
        
        start.append(cur['datetime'])
        end.append(next['datetime'])
    


    #new dataset with inside/outside and time start/end
    BeeTravel = pd.DataFrame({'location': loc, 'start': start, 'end': end}) 
    
    #classify short outside/insides as part of another group
    flight = ['inside'] * len(BeeTravel)
    
    #Classify short flights as inside-short
    for i in range(2,len(BeeTravel)-2):
        if BeeTravel['location'].iloc[i] != "Ramp" and (BeeTravel['end'].iloc[i] - BeeTravel['start'].iloc[i]).total_seconds() < 300:
            BeeTravel['location'].iloc[i] = BeeTravel['location'].iloc[i-2]
            if BeeTravel['location'].iloc[i] == "Inside":
                flight[i] = 'inside-short'

    #Classify outside flights depending on length
    for i in range(len(BeeTravel)):
        if BeeTravel['location'].iloc[i] == "Outside":
            if (BeeTravel['end'].iloc[i] - BeeTravel['start'].iloc[i]).total_seconds() < 600:
                flight[i] = "outside-short"
            elif (BeeTravel['end'].iloc[i] - BeeTravel['start'].iloc[i]).total_seconds() < 7200: #2 hrs
                flight[i] = "outside-foraging"
            else:
                flight[i] = "outside-long"


    BeeTravel['activity'] = flight
    return BeeTravel



Utilizes the previous function on every individual bee to have a full table.

In [6]:
def cleanData(data):

    #new dataset dict
    newdataset = {'tagID':[],'tripStart':[],'tripEnd':[]}
    activity = []
    #apply classifyloc to every bee ID
    for b in pd.unique(data['tagID']):
        dataset = classifyLoc(data[data['tagID'] == b])
        dataset = dataset.assign(tagID = b)
        activity.append(dataset)
        #for flights dataset
        for index, row in dataset.iterrows():
            if row['location'] == "Outside" and (row['activity'] == "outside-foraging" or row['activity'] == "outside-long"):
                newdataset['tagID'].append(b)
                newdataset['tripStart'].append(row['start'])
                newdataset['tripEnd'].append(row['end'])


    #all activity inside & outside
    allActivity = pd.concat(activity, axis = 0) 

    #all flights, same as all activity with only outside sections
    df = pd.DataFrame.from_dict(newdataset)
    df['duration'] = df['tripEnd'] - df['tripStart']

    return allActivity, df

Creates a statistical summary of an entire hive based on two tables, all activity and all flights.
Returns a table containing the statistical summary.

Does so by iterating over every beeID separately.

In [7]:
def summaryData(activity,flights):
    beeIDs = pd.unique(activity['tagID'])
    firstSeen = []
    lastSeen = []
    firstFlight = []
    lastFlight = []
    avgTrip = []
    medianTrip = []
    longestFlight = []
    shortestFlight = []
    numTrips = []
    tripsPerDay = []
    
    for b in beeIDs:
    
        indFlight = flights[flights['tagID'] == b].reset_index()
        
        beeDF = activity[activity['tagID'] == b].reset_index()
        firstSeen.append(beeDF['start'].iloc[0].replace(microsecond=0))    
        lastSeen.append(beeDF['end'].iloc[len(beeDF)-1].replace(microsecond=0))
        if len(indFlight) > 0:
            firstFlight.append(indFlight['tripStart'].iloc[0].replace(microsecond=0))
            lastFlight.append(indFlight['tripStart'].iloc[len(indFlight)-1].replace(microsecond=0))
        else:
            firstFlight.append(pd.NA)
            lastFlight.append(pd.NA)
        avgTrip.append(flights[flights['tagID'] == b]['duration'].mean())
        medianTrip.append(flights[flights['tagID'] == b]['duration'].median())
        numTrips.append(len(flights[flights['tagID'] == b]['duration']))

        #average flights per day
        indFlight['day'] = indFlight['tripStart'].apply(lambda x: x.date())
        #trips per day
        tpd = indFlight['day'].value_counts().sum()
        divisor = lastSeen[-1].day - firstSeen[-1].day + 1
        tripsPerDay.append(tpd/divisor)
        
        if len(flights[flights['tagID'] == b]) > 0:
             #longest flight per bee
            longest = flights[flights['tagID'] == b]['duration'].idxmax()
            longestLen = f'{int(flights.iloc[longest]['duration'].total_seconds()//3600)}:{int(flights.iloc[longest]['duration'].total_seconds()//60)%60}'
            longestFlight.append(f'{flights.iloc[longest]['tripStart'].replace(microsecond=0)} ~ {longestLen}')
            #shortest flight per bee
            shortest = flights[flights['tagID'] == b]['duration'].idxmin()
            shortestLen = f'{int(flights.iloc[shortest]['duration'].total_seconds()//3600)}:{int(flights.iloc[shortest]['duration'].total_seconds()//60)%60}'
            shortestFlight.append(f'{flights.iloc[longest]['tripStart'].replace(microsecond=0)} ~ {shortestLen}')
            
        else:
            longestFlight.append(pd.NA)
            shortestFlight.append(pd.NA)

    avgTrip = [f'{i.seconds//3600}:{(i.seconds//60)%60}' for i in avgTrip]
    
    beeDict = {"tagID":beeIDs, "First Seen":firstSeen, "Last Seen":lastSeen, 'First Flight':firstFlight, 'Last Flight':lastFlight, 'Longest Flight': longestFlight, 'Shortest Flight': shortestFlight, "Average Length of Flights":avgTrip, "# of Flights":numTrips, 'Average Flights per Day':tripsPerDay}
    beeSummary = pd.DataFrame.from_dict(beeDict)
    return beeSummary

Changes dates to fit xx/yy format instead of x/y for single digit numbers

In [8]:
def fixDate(x):
    if x.day > 9 and x.month > 9:
        return f'{x.month}/{x.day}'
    elif x.day > 9 and x.month < 10:
        return f'0{x.month}/{x.day}'
    elif x.day < 10 and x.month > 9:
        return f'{x.month}/0{x.day}'
    else:
        return f'0{x.month}/0{x.day}'
        

Apply functions and save CSV's

In [9]:
allActivity, flights = cleanData(dataset)
beeSummary = summaryData(allActivity,flights)
beeSummary
flights.to_csv("flights.csv")
beeSummary.to_csv("summary.csv")

Graph preparing functions

Obtain sunrise and sunset table for actograph based on previously defined timezone and required days for table.

In [10]:
def getSRSS(dataset, tz, lat, lon):
    dates = dataset['start'].apply(lambda x: x.date()).sort_values()
    dates = dates.unique()
    sunrise = []
    sunset = []
    sun = Sun(lat, lon)
    for d in dates:
        dT = datetime.datetime(d.year, d.month, d.day)
        sunrise.append(sun.get_sunrise_time(dT).astimezone(tz).time())
        sunset.append(sun.get_sunset_time(dT).astimezone(tz).time())

    srss = pd.DataFrame.from_dict({"date":dates,"sunrise":sunrise,"sunset":sunset})
    return srss
    

dt = getSRSS(allActivity, tz, lat, lon)
dt['date'] = dt['date'].apply(lambda x: fixDate(x))
print(dt)

    date   sunrise    sunset
0  06/21  05:49:12  19:03:00
1  06/22  05:49:12  19:03:36
2  06/23  05:49:48  19:03:36
3  06/24  05:49:48  19:03:36
4  06/25  05:49:48  19:04:12
5  06/26  05:50:24  19:04:12
6  06/27  05:50:24  19:04:12
7  06/28  05:51:00  19:04:12


Split overnight flights at midnight for plotting reasons

missing = True and timeam/timepm for when a dataset has missing values depending
on dataset requirement.

In [11]:
#fix up data
#separated by date for pseudo-actogram
def separateFlights(flights,missing=False,timeam="08:00:00",timepm="18:00:00"):
    newdata = []
    
    for i in range(len(flights)):
        row = flights.iloc[i].copy()
        row['day'] = row['tripStart'].to_pydatetime().day
        day = row['day']
        month0 = row['tripStart'].to_pydatetime().month
        month1 = row['tripEnd'].to_pydatetime().month
        
        if row['tripStart'].to_pydatetime().day != row['tripEnd'].to_pydatetime().day:
            #first day
            newentry = row.copy()
            newentry['tripEnd'] = row['tripEnd'].replace(hour=23,minute=59,second=59,day=1,month=1)
            newentry['tripStart'] = row['tripStart'].replace(day=1,month=1)
            newentry['date'] = fixDate(row['tripStart'])
            newdata.append(newentry)
            #second day
            newday = row['tripEnd'].to_pydatetime().day
            newentry = row.copy()
            newentry['tripStart'] = row['tripStart'].replace(day=1,hour=0,minute=0,second=0,month=1)
            newentry['tripEnd'] = row['tripEnd'].replace(day=1,month=1)
            newentry['day'] = newday
            newentry['date'] = fixDate(row['tripStart'])
            newdata.append(newentry)
        else:
            newentry = row.copy()
            newentry['tripStart'] = row['tripStart'].replace(day=1,month=1)
            newentry['tripEnd'] = row['tripEnd'].replace(day=1,month=1)
            newentry['date'] = fixDate(row['tripStart'])
            newdata.append(newentry)

    if missing:
        cutoffday = str(newentry['tripStart'].date())
        #Extra for missing dataset
        for i in range(len(newdata)):
            cutoffpm = datetime.strptime(cutoffday + " " + timepm,"%Y-%m-%d %H:%M:%S")
            cutoffam = datetime.strptime(cutoffday + " " + timeam,"%Y-%m-%d %H:%M:%S")
            if newdata[i]['tripEnd'] >= cutoffpm:
                newrow = newdata[i].copy()
                newdata[i]['tripEnd'] = cutoffpm
                newdata[i]['Data'] = 'Available'
                newrow['tripStart'] = cutoffpm
                newrow['Data'] = 'Missing'
                newdata.append(newrow)
            elif newdata[i]['tripStart'] <= cutoffam:
                newrow = newdata[i].copy()
                newdata[i]['tripStart'] = cutoffam
                newdata[i]['Data'] = 'Available'
                newrow['tripEnd'] = cutoffam
                newrow['Data'] = 'Missing'
                newdata.append(newrow)
            else:
                newdata[i]['Data'] = "Available"
    else:
        for i in range(len(newdata)):
            newdata[i]['Data'] = 'Available'
    
    
    flights_mod = pd.concat(newdata, axis = 1)
    flights_mod = flights_mod.transpose().reset_index()
    flights_mod = flights_mod.drop(["index"],axis=1)
    return flights_mod


Apply shapes to time graph based on sunrise/sunset dataset.

In [12]:
orange = "rgb(255, 199, 164)"
blue =  "rgb(199, 199, 255)"

In [13]:
def addShapes(fig, night_day_dataset, dataset):
    #all days must be set to the same first, with actual day and month on
    #day/month columns
    #obtain day thats set for all vals
    curday = str(dataset.iloc[0]['tripStart'].to_pydatetime().date())
    dates = dataset['date'].unique()
    #select first day and last day
    for i in range(len(dates)):
        #select row where day is day in iterator
        row = night_day_dataset[night_day_dataset['date'] == dates[i]]
        sunrise = row['sunrise'].values[0]
        sunset = row['sunset'].values[0]
        #set values for early morning rectangle
        x0 = curday + " 00:00:00"
        x1 = curday + " " + str(sunrise)
        y0 = i - 0.5
        y1 = i + 0.5
        fig.add_shape(type="rect",
            xref="x", yref="y",
            x0=x0, y0=y0,
            x1=x1, y1=y1,
            fillcolor=blue,
            line=dict(
                color=blue,
                width=0.5,
            ),
            layer="below"
         )
        #set values for day rectangle
        x0 = curday + " " + str(sunrise)
        x1 = curday + " " + str(sunset)
        fig.add_shape(type="rect",
            xref="x", yref="y",
            x0=x0, y0=y0,
            x1=x1, y1=y1,
            fillcolor=orange,
            line=dict(
                color=orange,
                width=0.5,
            ),
            layer="below"
         )
        #set values for night rectangle
        x0 = curday + " " + str(sunset)
        x1 = curday + " 23:59:59"
        fig.add_shape(type="rect",
            xref="x", yref="y",
            x0=x0, y0=y0,
            x1=x1, y1=y1,
            fillcolor=blue,
            line=dict(
                color=blue,
                width=0.5,
            ),
            layer="below"
         )

    times = [" 00:00:00"," 03:00:00", " 06:00:00", " 09:00:00", " 12:00:00", " 15:00:00", " 18:00:00", " 21:00:00", " 23:59:59"]

    for i in times:
        fig.add_vline(x=curday + i, line_width=0.5, line_dash="dash", line_color="black")

    for i in range(len(dates)-1):
        fig.add_hline(y=i+0.5, line_width=0.5, line_dash="dash", line_color="black")
    

    
    

Create time plot for bee flights utilizing flights dataset.

In [14]:
def createActoGraph(dataset):
    fig = px.timeline(dataset, x_start="tripStart", x_end="tripEnd",y="date", title="Selected Bee's Flights Over Time")
    fig.update_traces(
        marker_line_color="grey",   
        marker_line_width=1          
    )
    fig.update_xaxes(showgrid=False)
    fig.update_yaxes(showgrid=False)
    #change tick format to look better
    fig.update_layout(xaxis=dict(
                      title='Time', 
                      tickformat = '%H:%M:%S'),
                     plot_bgcolor="white",
    #annotations to make pseudo legend
    annotations=[
        dict(
            x=1.095,
            y=1.1,
            xref="paper",
            yref="paper",
            showarrow=False,
            text="Time of Day",
            align="left",
            font=dict(size=13,family="Open Sans",color="black")
        )
    ])
    #setup axes
    fig.update_yaxes(dtick=1)
    fig.update_yaxes(autorange="reversed",title="Day")
    vals = dataset['date'].unique()
    fig.update_yaxes(
        tickvals=vals
    )
    fig.update_traces(marker_color='brown')

    #custom legend
    custom_legend_items = [
        dict(name="Day", color=orange, symbol="square"),
        dict(name="Night", color=blue, symbol="square"),
    ]

    addShapes(fig,dt,dataset)

    for item in custom_legend_items:
        fig.add_trace(go.Scatter(
            x=[None], y=[None],
            mode="markers",
            marker=dict(size=10, color=item["color"], symbol=item["symbol"]),
            name=item["name"],
            legendgroup="custom",
            showlegend=True
        ))

    
    fig.show()


Plot histogram displaying number of flights at time of day and number of flights per day
of recorded data.

In [15]:
def plotHist(dataset):
    dataset['hour'] = dataset['tripStart'].apply(lambda x: x.hour)
    dataset['day'] = dataset['tripStart'].apply(lambda x: x.day)
    fig = sns.histplot(dataset, x='hour',bins= (max(dataset['hour']) - min(dataset['hour'])), color="brown",zorder=3)
    
    vals = range(min(dataset['hour']), max(dataset['hour'])+1)
    text = [(str(xi) + ":00:00") for xi in vals]

    fig.set_xticks(vals) 
    fig.set_xticklabels(text)
    fig.yaxis.set_major_locator(mticker.MaxNLocator(integer=True))
 
    plt.title("Number of Flights at Time of Day.")
    plt.xticks(rotation=30)
    plt.grid()
    fig.set(xlabel='Hour', ylabel='Count of flights')
    plt.show()
    fig = fig.get_figure()
    fig.savefig("Hist1.png") 

    dataset['date'] = dataset['tripStart'].apply(lambda x: fixDate(x))
    fig = sns.histplot(dataset, x='date',bins=(max(dataset['day']) - min(dataset['day'])), color="brown",zorder=3)
    fig.yaxis.set_major_locator(mticker.MaxNLocator(integer=True))
    plt.xticks(dataset['date'].unique())
    fig.set(xlabel='Day', ylabel='Number of flights')
    plt.title("Number of Flights Per Day")
    plt.grid()
    plt.show()

    fig = fig.get_figure()
    fig.savefig("Hist2.png") 

In [16]:
flights_mod = separateFlights(flights)
flights_mod

,tagID,tripStart,tripEnd,duration,day,date,Data
0,1621,2017-01-01 08:10:33.850000,2017-01-01 08:24:48.250000,0 days 00:14:14.400000,21,06/21,Available
1,1621,2017-01-01 08:55:48.300000,2017-01-01 09:12:41.900000,0 days 00:16:53.600000,21,06/21,Available
2,1621,2017-01-01 09:27:15.300000,2017-01-01 10:03:14.850000,0 days 00:35:59.550000,21,06/21,Available
3,1621,2017-01-01 10:48:33.700000,2017-01-01 11:35:05.750000,0 days 00:46:32.050000,21,06/21,Available
4,1621,2017-01-01 11:43:25.300000,2017-01-01 12:41:51.500000,0 days 00:58:26.200000,21,06/21,Available
...,...,...,...,...,...,...,...
226,1722,2017-01-01 10:14:09.050000,2017-01-01 11:24:16.650000,0 days 01:10:07.600000,27,06/27,Available
227,1722,2017-01-01 11:26:43,2017-01-01 12:57:40.450000,0 days 01:30:57.450000,27,06/27,Available
228,1722,2017-01-01 15:23:10.050000,2017-01-01 15:33:57.450000,0 days 00:10:47.400000,27,06/27,Available
229,661,2017-01-01 13:57:57.300000,2017-01-01 16:13:55.650000,0 days 02:15:58.350000,22,06/22,Available


In [17]:
#function to change hour column into hour properly

def fixTime(hr):
    if hr < 10:
        return "0" + str(hr) + ":00"
    else:
        return str(hr) + ":00"

## Widget with bee selector

All previous functions to generate full display

In [18]:
output2 = widgets.Output()
from matplotlib.colors import LinearSegmentedColormap

def on_change(change):
    bee = dataset[dataset['tagID'] == selectBee.value].copy().reset_index()
    beeflights = flights[flights['tagID'] == selectBee.value].copy().reset_index()
    beemod = flights_mod[flights_mod['tagID'] == selectBee.value].copy().reset_index()
    beeSum = beeSummary.copy()
    
    with output2:
        output2.clear_output() 
        if len(beeflights) > 0: 
            createActoGraph(beemod)       
            plotHist(beeflights)
            beeflights['date'] = beeflights['tripStart'].apply(lambda x: fixDate(x))
            #heatmap with date and hour for single bee
            beeflights['hour'] = beeflights['hour'].apply(lambda x: fixTime(x))
            table = beeflights.pivot_table(index="date", columns="hour", values="tripStart", aggfunc='count')
            cmap = sns.color_palette("Reds", int(table.max().max()))
            fig = sns.heatmap(table, cmap=cmap)
            fig.set(xlabel='Hour of Day', ylabel='Day')
            colorbar = fig.collections[0].colorbar
            colorbarTicks = np.arange(1,table.max().max() + 1)
            colorbar.set_ticks(colorbarTicks)
            colorbar.update_ticks()
            plt.title("Number of flights at time of day")
            plt.show()

            #lineplot of flight duration mean
            durationmean = beeflights.drop('hour',axis=1).groupby(['date','tagID']).mean().reset_index()
            durationmean['duration'] = durationmean['duration'].apply(lambda x : x.total_seconds()/60)
            fig = sns.lineplot(data=durationmean,x='date',y='duration',marker="o",color="brown")
            fig.set(xlabel='Date', ylabel='Average Flight Duration (minutes)')
            plt.title("Average flight duration across days for selected bee")
            plt.xticks(rotation=30)
            plt.show()


        
        else:
            print("=======================")
            print("Bee has no flight data.")
            print("=======================")

        print("\n Summary of selected bee:")
        display(beeSum[beeSum['tagID'] == selectBee.value])
        

selectBee = widgets.Dropdown(
    options=pd.unique(dataset['tagID']),
    description='Bee Selector:',
    value=None,
    disabled=False,
)

selectBee.observe(on_change, names='value')

display(selectBee,output2)

Dropdown(description='Bee Selector:', options=(1621, 516, 677, 638, 197, 1651, 1361, 1652, 1609, 1109, 1786, 1…

Output()